# End Child Poverty Act

A simulation of the End Child Poverty Act and its possible effects (TODO: make a nice paragraph describing what we're doing here)

In [1]:
from openfisca_us import Microsimulation, IndividualSim

In [2]:
from openfisca_us.model_api import *
from openfisca_us.variables.household.demographic.tax_unit.filing_status import (
    FilingStatus,
)
from openfisca_us.tools.baseline_variables import baseline_variables

REPLACED_PROGRAMS = ["ctc", "eitc"]

CHILD_AMOUNT = 393
CHILD_AGE_LIMIT = 18
ADULT_DEPENDENT_CREDIT = 600
SINGLE_FILER_CREDIT = 600
MARRIED_FILER_CREDIT = 1200
PHASE_OUT_START_SINGLE = 20_000
PHASE_OUT_START_MARRIED = 40_000
PHASE_OUT_RATE = 0.05

class ecpa(Variable):
    value_type = float
    entity = TaxUnit
    definition_period = YEAR
    label = "End Child Poverty Act"

    def formula(tax_unit, period, parameters):
        person = tax_unit.members
        qualifies_for_child_allowance = person("age", period) <= CHILD_AGE_LIMIT
        child_allowance = CHILD_AMOUNT * tax_unit.sum(qualifies_for_child_allowance)

        # TODO: check if 18 year old tax filer or spouse of household is eligible
        adult_dependent = person("is_tax_unit_dependent", period) & ~qualifies_for_child_allowance
        dependent_credit = ADULT_DEPENDENT_CREDIT * tax_unit.sum(adult_dependent)

        filing_status = tax_unit("filing_status", period)
        is_married = filing_status == filing_status.possible_values.JOINT
        max_filer_credit = where(is_married, MARRIED_FILER_CREDIT, SINGLE_FILER_CREDIT)

        agi = tax_unit("adjusted_gross_income", period)
        phase_out_start = where(is_married, PHASE_OUT_START_MARRIED, PHASE_OUT_START_SINGLE)

        excess = max_(agi - phase_out_start, 0)
        reduction = excess * PHASE_OUT_RATE
        filer_credit = max_(max_filer_credit - reduction, 0)

        return (child_allowance * 12) + dependent_credit + filer_credit

class income_tax(Variable):
    def formula(tax_unit, period, parameters):
        original = baseline_variables["income_tax"].formula(tax_unit, period)
        return original - tax_unit("ecpa", period)

class add_ecpa(Reform):
    def apply(self):
        for program in REPLACED_PROGRAMS:
            self.neutralize_variable(program)
        self.add_variable(ecpa)
        self.update_variable(income_tax)

In [3]:
def make_tax(adults, children, reform):
    if reform is None:
        sim = IndividualSim(year=2022)
    else:
        sim = IndividualSim(reform, year=2022)
    
    sim.add_person(name="head", age=25)
    members = ["head"]
    if adults == 2:
        sim.add_person(name="spouse", age=25)
        members += ["spouse"]
    for i in range(children):
        child = "child{}".format(i)
        sim.add_person(name=child, age=6)
        members += [child]
    sim.add_tax_unit(name="tax_unit", members=members, premium_tax_credit=0)
    sim.add_spm_unit(name="spm_unit", members=members, snap_emergency_allotment=0)
    sim.add_household(name="household", members=members, state_code="MA")

    sim.vary("employment_income", max=100_000, step=100)

    employment_income = sim.calc("employment_income")[0]
    spm_unit_net_income = sim.calc("spm_unit_net_income")[0].round()
    mtr = 1 - sim.deriv(
        "spm_unit_net_income", "employment_income", wrt_target="head"
    )
    
    return pd.DataFrame(
        dict(
            employment_income=employment_income,
            spm_unit_net_income=spm_unit_net_income,
            mtr=mtr,
            adults=adults,
            children=str(children),
            scenario="Baseline" if reform is None else "ECPA",
        )
    )

In [18]:
import pandas as pd

l = []
for adults in range(1, 3):
    for children in range(0, 4):
        for reform in [None, add_ecpa]:
            l.append(make_tax(adults, children, reform))

df = pd.concat(l)

LABELS = dict(
    employment_income="Employment income",
    spm_unit_net_income="Net income",
    mtr="Marginal tax rate",
    adults="Adults",
    children="Children",
    scenario="Scenario"
)
df

,employment_income,spm_unit_net_income,mtr,adults,children,scenario
0,0.0,3000.0,-0.022949,1,0,Baseline
1,100.0,3102.0,-0.022952,1,0,Baseline
2,200.0,3205.0,-0.022949,1,0,Baseline
3,300.0,3307.0,-0.022949,1,0,Baseline
4,400.0,3409.0,-0.022952,1,0,Baseline
...,...,...,...,...,...,...
996,99600.0,93766.0,0.246484,2,3,ECPA
997,99700.0,93841.0,0.246562,2,3,ECPA
998,99800.0,93916.0,0.246484,2,3,ECPA
999,99900.0,93992.0,0.246484,2,3,ECPA


In [16]:
wide = df.pivot_table(index=["adults", "children", "employment_income"], columns="scenario", values=["mtr", "spm_unit_net_income"]).reset_index()

wide["diff"] = wide["spm_unit_net_income"]["ECPA"] - wide["spm_unit_net_income"]["Baseline"]
wide["mtr_diff"] = wide["mtr"]["ECPA"] - wide["mtr"]["Baseline"]
wide

adults children employment_income       mtr            \
scenario                                    Baseline      ECPA   
0             1        0               0.0 -0.022949  0.076499   
1             1        0             100.0 -0.022952  0.076501   
2             1        0             200.0 -0.022949  0.076499   
3             1        0             300.0 -0.022949  0.076501   
4             1        0             400.0 -0.022952  0.076499   
...         ...      ...               ...       ...       ...   
8003          2        3           99600.0  0.246484  0.246484   
8004          2        3           99700.0  0.246562  0.246562   
8005          2        3           99800.0  0.246484  0.246484   
8006          2        3           99900.0  0.246484  0.246484   
8007          2        3          100000.0  0.246484  0.246484   

         spm_unit_net_income             diff  mtr_diff  
scenario            Baseline     ECPA                    
0                     3000.0   3600.0   600.0  0.099448  
1                     3102.0   3692.0   590.0  0.099453  
2                     3205.0   3785.0   580.0  0.099448  
3                     3307.0   3877.0   570.0  0.099451  
4                     3409.0   3969.0   560.0  0.099451  
...                      ...      ...     ...       ...  
8003                 85618.0  93766.0  8148.0  0.000000  
8004                 85693.0  93841.0  8148.0  0.000000  
8005                 85768.0  93916.0  8148.0  0.000000  
8006                 85844.0  93992.0  8148.0  0.000000  
8007                 85919.0  94067.0  8148.0  0.000000  

[8008 rows x 9 columns]

In [21]:
import plotly.express as px

from ubicenter import format_fig

fig = px.line(
    wide,
    "employment_income",
    "diff",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Difference between Baseline vs. ECPA",
)

format_fig(fig)

In [6]:
px.line(df, x="employment_income", y="diff")

ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['employment_income', 'spm_unit_net_income', 'mtr', 'adults', 'children', 'scenario'] but received: diff

In [ ]:
fig = px.line(
    df,
    x="employment_income",
    y=["mtr_baseline", "mtr_ecpa"]
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat=".1%",
    yaxis_range=[-1, 1],
)
fig.show()

In [ ]:
px.line(df, x="employment_income", y="mtr_diff")